# Read aggregated results 
The collect_results.py script aggregates results from multiple directories. You can view and manipulate the aggregated results to get insights from the data

In [1]:
import glob
import os
import json
import re
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import time

import pandas as pd
from pandas import DataFrame

import sys
# add reference to the few_shot dir
path_base = os.path.abspath(os.path.join(sys.path[0], '../'))
print(path_base)
sys.path.insert(1, path_base)
from collect_results import view_preferred_metrics, get_first_candidates_prompts, get_first_pred_prompts_example

/private/home/tbmihaylov/fairseq-xlmg/examples/few_shot


Read the raw results file:

In [2]:
# This is the path to the raw results file generated with the script
results_json = "/private/home/tbmihaylov/fairseq-xlmg/gpt3_repro_cb_results.tsv.raw.jsonl"
results_json = "/private/home/tbmihaylov/fairseq-xlmg/cb_our_models.tsv.raw.jsonl"
#results_json = "/checkpoint/tbmihaylov/few_shot/xnli_experimental/results_200.tsv.raw.jsonl"

# python examples/few_shot/scripts/collect_results.py -i /checkpoint/tbmihaylov/few_shot/xnli_experimental/*smpl.All* -o /checkpoint/tbmihaylov/few_shot/xnli_experimental/results.tsv -v preferred_metrics_mean
results_json = "/checkpoint/tbmihaylov/few_shot/xnli_experimental/results.tsv.raw.jsonl"

#python examples/few_shot/scripts/collect_results.py -i /checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1 -o /checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1/results.tsv -v preferred_metrics_mean
results_json = "/checkpoint/tbmihaylov/few_shot/multilingual_xnli_1.3B_gpt3_setting_checkpoints/results.tsv.raw.jsonl"
# python examples/few_shot/scripts/collect_results.py -i /checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1_full -o /checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1_full/results.tsv -v preferred_metrics_mean
results_json = "/checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1_full/results.tsv.raw.jsonl"
#results_json = "/checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1_full_calib/results.tsv.raw.jsonl"

In [3]:
only_monolingual = True 

results = []
with open(results_json) as f_res:
    for line in f_res:
        line = line.strip()
        item = json.loads(line)
        if 'run_params::n_eval_samples' not in item:
            item['run_params::n_eval_samples'] = 0
            
        if only_monolingual and item["train_lang"] != item["language"]:
            continue
        results.append(item)

print(f"{len(results)} items loaded")
#print(item)

450 items loaded


In [4]:
df = DataFrame.from_records(results)

The results has the followinf columns:

In [5]:
list(df.columns)

['model_name',
 'task',
 'language',
 'template',
 'nb_few_shot_samples',
 'calibration_options',
 'calibrator_name',
 'train_set',
 'valid_set',
 'eval_set',
 'train_lang',
 'valid_lang',
 'ppl_common_prefix::scores',
 'ppl_common_prefix::mean',
 'ppl_common_prefix::std',
 'ppl_common_prefix::mean_confidence_interval',
 'ppl_selected_candidate::scores',
 'ppl_selected_candidate::mean',
 'ppl_selected_candidate::std',
 'ppl_selected_candidate::mean_confidence_interval',
 'ppl_full_selected_candidate::scores',
 'ppl_full_selected_candidate::mean',
 'ppl_full_selected_candidate::std',
 'ppl_full_selected_candidate::mean_confidence_interval',
 'ppl_candidates_full_prompt__entailment::scores',
 'ppl_candidates_full_prompt__entailment::mean',
 'ppl_candidates_full_prompt__entailment::std',
 'ppl_candidates_full_prompt__entailment::mean_confidence_interval',
 'ppl_candidates_full_prompt__contradiction::scores',
 'ppl_candidates_full_prompt__contradiction::mean',
 'ppl_candidates_full_prompt_

## Print template verbalized prompts

In [6]:
print("Verbalized examples:")
print("#" * 10)
for item in results:
    if item["language"] == "en":
        verbalized_prompt = get_first_pred_prompts_example(item["results_file"])
        print(f"task:{item['task']} template: {item['template']}:\n{verbalized_prompt}")

Verbalized examples:
##########
task:xnli template: xnli_generativenli_sentence_mt:
### Prompt for cand `entailment`:
And he said, Mama, I'm home. Right? Yes, he called his mom as soon as the school bus dropped him off.
##########
task:xnli template: xnli_generativenli_mt:
### Prompt for cand `entailment`:
And he said, Mama, I'm home, right? Yes, he called his mom as soon as the school bus dropped him off.
##########
task:xnli template: xnli_generativenli_sentence__en:
### Prompt for cand `entailment`:
And he said, Mama, I'm home.. Right? Yes, he called his mom as soon as the school bus dropped him off.
##########
task:xnli template: xnli_generativenli__en:
### Prompt for cand `entailment`:
And he said, Mama, I'm home, right? Yes, he called his mom as soon as the school bus dropped him off.
##########
task:xnli template: generativenli:
### Prompt for cand `entailment`:
And he said, Mama, I'm home, right? Yes, he called his mom as soon as the school bus dropped him off.
##########
task:

# Create a custom view using pandas
You can use the pandas api to manipulate the table and create custom views. 

In [9]:
def view_preferred_metrics_mean(df):
    run_columns = ["task", "eval_set", "run_params::n_eval_samples", "language", "train_set", "train_lang", "template", "nb_few_shot_samples", "calibration", "run_params::scoring", "model_name"]
    suffixes = ["::mean"]
    
    return view_preferred_metrics(df, run_columns, suffixes)

df_selected = view_preferred_metrics_mean(df)

value_cols = "accuracy::mean"
index_cols = ["model_name", "calibration", "task", "eval_set", "run_params::n_eval_samples", "train_set", "nb_few_shot_samples", 
              "template",
             ]
cols = ["language"]
pt = pd.pivot_table(df_selected, values=value_cols, index=index_cols,
                    columns=cols, aggfunc=[np.mean])
#pt = pt.swaplevel(0, 1, axis=1).sort_index(axis=1)

output_tsv = results_json + ".formatted.tsv"
pt.to_csv(output_tsv, sep="\t")
print(output_tsv)
pt

/checkpoint/tbmihaylov/few_shot/2021_09_multilingual_eval_v1_full/results.tsv.raw.jsonl.formatted.tsv


mean  \
language                                                                                                                                                                      ar   
model_name                                         calibration task eval_set run_params::n_eval_samples train_set nb_few_shot_samples template                                     
l28_64e_1b24_top1_cc100_combined_roberta           False       xnli dev      0                          test      0                   generativenli                    41.445783   
                                                                                                                                      xnli_generativenli__en           41.445783   
                                                                                                                                      xnli_generativenli_mt            33.895582   
                                                                                                                                      xnli_generativenli_sentence__en  42.610442   
                                                                                                                                      xnli_generativenli_sentence_mt   35.622490   
l28_dense_1b24_2cc100_combined_roberta             False       xnli dev      0                          test      0                   generativenli                    46.265060   
                                                                                                                                      xnli_generativenli__en           46.265060   
                                                                                                                                      xnli_generativenli_mt            33.614458   
                                                                                                                                      xnli_generativenli_sentence__en  44.899598   
                                                                                                                                      xnli_generativenli_sentence_mt   35.381526   
l28hi_64e_top2_l24                                 False       xnli dev      0                          test      0                   generativenli                    40.883534   
                                                                                                                                      xnli_generativenli__en           40.883534   
                                                                                                                                      xnli_generativenli_mt            32.891566   
                                                                                                                                      xnli_generativenli_sentence__en  41.204819   
                                                                                                                                      xnli_generativenli_sentence_mt   35.261044   
moe_64exp_lang1_cc100_combined_roberta_0921        False       xnli dev      0                          test      0                   generativenli                    35.020080   
                                                                                                                                      xnli_generativenli__en           35.020080   
                                                                                                                                      xnli_generativenli_mt            33.092369   
                                                                                                                                      xnli_generativenli_sentence__en  34.979920   
                                                                                                                                      xnli_generativenli_sentence_mt   34.497992   
moe_64exp_lang32_cc100_combined_

# Make your view available to others
If you think that you created a view that might be useful to others, you can add it to the collect_results.py.
Simply add the `my_custom_view` function to the `display_views` dictionary in [collect_results.py](examples/few_shot/scripts/collect_results.py) and the custom view will be available as -v my_custom_view_freindly.

In [ ]:
#df[df["run_params::train_sep"] == "\n\n"]